# ZaloAI Lyrics Alignment Predict

In [27]:
import os
import json
from time import perf_counter

from tqdm import tqdm
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC

from predict_utils import end_to_end_align, load_single_example
from segment_utils import load_annotate_lyrics, do_segmentation_transform

cache_dir = './cache/'
processor = Wav2Vec2Processor.from_pretrained("nguyenvulebinh/wav2vec2-base-vietnamese-250h", cache_dir=cache_dir)
model = Wav2Vec2ForCTC.from_pretrained("model", cache_dir=cache_dir)

SONGS_PATH = "/public_test/songs"
LYRICS_PATH = "/public_test/lyrics"

file_ids = os.listdir(SONGS_PATH)
file_ids = [file.split(".")[0] for file in file_ids]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [28]:
start_time = perf_counter()

In [4]:
# Predict alignment for each input song
results = []
for file in tqdm(file_ids):
    song = load_single_example(file)
    alignment = end_to_end_align(song, processor, model)

    results.append({
        "song": file,
        "alignment": alignment
    })

# Do sentence segmentation
annotate_lyrics = load_annotate_lyrics(file_ids)
s = do_segmentation_transform(results, annotate_lyrics)

100%|█████████████████████████████████████████| 264/264 [16:53<00:00,  3.84s/it]

CPU times: user 1h 22min 5s, sys: 4min 52s, total: 1h 26min 58s
Wall time: 16min 53s


In [26]:
if not os.path.exists("submission"):
    os.mkdir("submission")

for key, data in s.items():
    with open(os.path.join("submission", key + ".json"), "wt") as f:
        json.dump(data, f, ensure_ascii=False)

In [29]:
end_time = perf_counter()

In [30]:
elapsed_inference_time = end_time - start_time

In [31]:
elapsed_inference_time

6.927912546030711